In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 2s (140 kB/s)
Reading package li

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-25 20:03:53--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.6’

postgresql-42.2.16. 100%[===================>] 979.38K  4.74MB/s    in 0.2s    

2022-06-25 20:03:54 (4.74 MB/s) - ‘postgresql-42.2.16.jar.6’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("prediction").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Import relevant libraries

from google.colab.patches import cv2_imshow
from google.colab.patches import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from tensorflow.keras.models import load_model
import base64
from imageio import imread
import io
from PIL import Image
import os

from sqlalchemy import create_engine

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
training_data_file_Path = "/content/drive/MyDrive/Model_Generator_Input/"
training_data_input_file = "A_Z Handwritten Data Added.csv"
file_to_process = training_data_file_Path + training_data_input_file

In [7]:
# # Read the dataset
# # Read the data as data type float32 as the csv file is very large

training_data_df = pd.read_csv(file_to_process, header=None)
training_data_df.columns = [str(i) for i in range(len(training_data_df.columns)) ]
training_data_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
training_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373881 entries, 0 to 373880
Columns: 785 entries, 0 to 784
dtypes: int64(785)
memory usage: 2.2 GB


In [9]:
# Function to predict letter characters using the trained model
xx = []
def generate_training_data(in_read_image):

    # read image
    image = cv2.imread(in_read_image)
    # make copy of original image, copy will be used to change colors
    image_copy = image.copy()

    # convert image to RGB using cvtColor
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Resize image to 400 x 400 size
    image = cv2.resize(image, (400,400))

    # Add blur to image and greyscale (need to greyscale as cv.threshold needs greyscale images)
    image_copy = cv2.GaussianBlur(image_copy, (7,7), 0)
    grey_image = cv2.cvtColor(image_copy, cv2.COLOR_BGR2GRAY)


    # Separate object from background pixels using thresholding
    # https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html
    _, img_thresh = cv2.threshold(grey_image, 100, 255, cv2.THRESH_BINARY_INV)

    # Resize and reshape image to fit trained_model requirements
    final_image = cv2.resize(img_thresh, (784,1))


    return(final_image)

In [10]:
def convert_image_to_base64(in_image_file_name):
# Function that takes in image file name reads in mage and convert to a base 64 string
  file_name = (in_image_file_name)
  with open(file_name, "rb") as img_file:
      b64_string = base64.b64encode(img_file.read())

  return(b64_string.decode('utf-8'))

In [11]:
def convert_from_base_64_to_image(in_base_64_image_string):
# Function that takes in base 64 string and converts to image
  image_webP = Image.open((io.BytesIO(base64.b64decode(in_base_64_image_string))))

  image_RGB = image_webP.convert("RGB")
  
  image = np.float32(image_RGB)

  return(image)

In [12]:
def generate_new_training_data_file_name(in_original_file_name):

  if in_original_file_name == "":
    in_original_file_name = "NEW_TRAINING_DATA.csv"
  current_date_time = dt.datetime.now(dt.timezone.utc).strftime('%d_%m_%Y_%H_%M_%S')

  split_in_original_file_name = os.path.splitext(in_original_file_name)

  file_name = split_in_original_file_name[0]
  file_extension = split_in_original_file_name[1]

  new_file_name = file_name + "_" + current_date_time+ file_extension

  return(new_file_name)

In [13]:
# Read and image and covert to base 64

# Image to use for conversion and splitting

directory_path = '/content/drive/MyDrive/Resized_images/'
directory_listing = os.listdir(directory_path)

new_training_data_df = pd.DataFrame()

for image_name in directory_listing:

    image_file_name = directory_path + image_name
    # Determine image character from file name A = img011-xxx.png Z = img036-xxx.png : Formula to determine ASCII value : (int(image_name[4:6]) - 11) + 65 : Convert ASCII value to character : chr(ascii_value)
    print(image_name)
    image_reference_integer = int(image_name[4:6])
    image_character = (image_reference_integer-11) + 65

    # Value for training data column 
    image_training_reference = image_reference_integer - 11 

    # Generate image in base 16 values 

    image_array = generate_training_data(image_file_name)

    new_training_data_entry_df = pd.DataFrame(image_array)

    # Add a new column to the beginning of the dataframe
    new_training_data_entry_df.insert(0, 'New Column',image_training_reference )

    # Rename the columns of the dataframe starting at 0
    new_training_data_entry_df.columns = [str(i) for i in range(len(new_training_data_entry_df.columns)) ]
    new_training_data_df = new_training_data_df.append(new_training_data_entry_df,ignore_index=True)
    # new_training_data_entry_df.head()


img011-new.png
img014-NEW.png
img013-new.png
img017-new.png
img023-new.png
img030-new.png
img025-new.png
img036-new.png


In [14]:
# Read the training data from either CSV file Database based on a parameter value training_data_source , for CSV value is set to string "CSV" , from DATABASE value is set to "DATABASE"

# Create log reference
validation_run_reference = dt.datetime.now(dt.timezone.utc)

training_data_source = "DATABASE"
if training_data_source == "CSV":
    # Append training data generate to original dataset
    training_data_df = training_data_df.append(new_training_data_df,ignore_index=True)
    output_file_name = generate_new_training_data_file_name(training_data_input_file)
    file_to_generate = training_data_file_Path + output_file_name
    training_data_df.to_csv(file_to_generate, index = False , header = False)
elif training_data_source == "DATABASE":
    # Convert dataframe to sparks dataframe , easier to write to database
    df_to_write_spark = spark.createDataFrame(new_training_data_df)
    # Configure settings for RDS
    mode = "append"
    database_table_name = "a_z_handwritten_data"
    df_to_write_spark = spark.createDataFrame(new_training_data_df)
    jdbc_url="jdbc:postgresql://finalpostgresdb.cxwdymdhaxq6.us-east-1.rds.amazonaws.com:5432/my_final_project"
    config = {"user":"root", "password": "Open4039!", "driver":"org.postgresql.Driver"}
    df_to_write_spark.write.jdbc(url=jdbc_url, table=database_table_name, mode=mode, properties=config)
else:
    print("no training datasource is set , no file written")